### 1. Installation

In [ ]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q http://apache.osuosl.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark


Download and install Apache Spark 3.2.1, along with its dependencies (Java Development Kit 8+)

Set Environment Variables to make the Spark runtime visible to Linux OS. Please, note that you can manage multiple versions of spark by pointing to the correct version through environment variables. Run below set of commands to point to Apache Spark 3.2.1 version downloaded earlier.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

Test the installation worked successfully.

In [ ]:
import findspark
findspark.init()

In [ ]:
# Verify the Spark version running on the virtual cluster
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

assert  "3." in sc.version, "Verify that the cluster Spark's version is 3.x"

If the environment is not configured properly, the instruction will show the Spark's version that would not be the 3.0.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
print(spark)

In [ ]:
#Collegamento al Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

#Pyspark

In [ ]:
with open("/content/drive/MyDrive/KPMG/RegressionDataset.xlsx", 'rb') as fp:
    df = pd.read_excel(fp)


In [ ]:
df

,AnnoAccademico,Ateneo,PopolazioneP,AteneiP,PopolazioneR,AteneiR,Fuorisede,Immatricolati
0,2010/2011,Aosta,126957,1,126957,1,31,188
1,2011/2012,Aosta,127153,1,127153,1,21,172
2,2012/2013,Aosta,127305,1,127305,1,31,173
3,2013/2014,Aosta,127951,1,127951,1,54,192
4,2014/2015,Aosta,128245,1,128245,1,52,181
...,...,...,...,...,...,...,...,...
644,2016/2017,Verona,918050,1,4890648,4,2509,4553
645,2017/2018,Verona,918069,1,4883373,4,2477,4484
646,2018/2019,Verona,919179,1,4880936,4,2098,3874
647,2019/2020,Verona,922857,1,4884590,4,2082,3785


In [ ]:
df=df.to_dict('records')

In [ ]:
import json

path = "/content/drive/MyDrive/KPMG/"

with open(path + 'RegressionDataset.json', 'w') as jsonfile:
    json.dump(df, jsonfile)

In [ ]:
# Read JSON file into dataframe
path = "/content/drive/MyDrive/KPMG/"

df_spark = spark.read.json(path + 'RegressionDataset.json')
df_spark.printSchema()
df_spark.show()

root
 |-- AnnoAccademico: string (nullable = true)
 |-- AteneiP: long (nullable = true)
 |-- AteneiR: long (nullable = true)
 |-- Ateneo: string (nullable = true)
 |-- Fuorisede: long (nullable = true)
 |-- Immatricolati: long (nullable = true)
 |-- PopolazioneP: long (nullable = true)
 |-- PopolazioneR: long (nullable = true)

+--------------+-------+-------+------+---------+-------------+------------+------------+
|AnnoAccademico|AteneiP|AteneiR|Ateneo|Fuorisede|Immatricolati|PopolazioneP|PopolazioneR|
+--------------+-------+-------+------+---------+-------------+------------+------------+
|     2010/2011|      1|      1| Aosta|       31|          188|      126957|      126957|
|     2011/2012|      1|      1| Aosta|       21|          172|      127153|      127153|
|     2012/2013|      1|      1| Aosta|       31|          173|      127305|      127305|
|     2013/2014|      1|      1| Aosta|       54|          192|      127951|      127951|
|     2014/2015|      1|      1| Aosta| 

In [ ]:
df_spark=df_spark.na.drop()


In [ ]:
from pyspark.sql.functions import col, split
df = df_spark.withColumn("Ateneo_Array", split(col("Ateneo")," "))
df = df.withColumn('AnnoAccademico_Array', split(col("AnnoAccademico")," "))
df.show()

+--------------+-------+-------+------+---------+-------------+------------+------------+------------+--------------------+
|AnnoAccademico|AteneiP|AteneiR|Ateneo|Fuorisede|Immatricolati|PopolazioneP|PopolazioneR|Ateneo_Array|AnnoAccademico_Array|
+--------------+-------+-------+------+---------+-------------+------------+------------+------------+--------------------+
|     2010/2011|      1|      1| Aosta|       31|          188|      126957|      126957|     [Aosta]|         [2010/2011]|
|     2011/2012|      1|      1| Aosta|       21|          172|      127153|      127153|     [Aosta]|         [2011/2012]|
|     2012/2013|      1|      1| Aosta|       31|          173|      127305|      127305|     [Aosta]|         [2012/2013]|
|     2013/2014|      1|      1| Aosta|       54|          192|      127951|      127951|     [Aosta]|         [2013/2014]|
|     2014/2015|      1|      1| Aosta|       52|          181|      128245|      128245|     [Aosta]|         [2014/2015]|
|     20

In [ ]:
#Import Spark CountVectorizer
from pyspark.ml.feature import CountVectorizer

# Initialize a CountVectorizer.
colorVectorizer = CountVectorizer(inputCol="Ateneo_Array", outputCol="Ateneo_OneHotEncoded", vocabSize=4, minDF=1.0)

#Get a VectorizerModel
colorVectorizer_model = colorVectorizer.fit(df)
df_ohe = colorVectorizer_model.transform(df)

# Initialize a CountVectorizer.
colorVectorizer = CountVectorizer(inputCol="AnnoAccademico_Array", outputCol="AnnoAccademico_OneHotEncoded", vocabSize=4, minDF=1.0)

#Get a VectorizerModel
colorVectorizer_model = colorVectorizer.fit(df)
df_ohe = colorVectorizer_model.transform(df_ohe)
df_ohe.show(truncate=False)


+--------------+-------+-------+------+---------+-------------+------------+------------+------------+--------------------+--------------------+----------------------------+
|AnnoAccademico|AteneiP|AteneiR|Ateneo|Fuorisede|Immatricolati|PopolazioneP|PopolazioneR|Ateneo_Array|AnnoAccademico_Array|Ateneo_OneHotEncoded|AnnoAccademico_OneHotEncoded|
+--------------+-------+-------+------+---------+-------------+------------+------------+------------+--------------------+--------------------+----------------------------+
|2010/2011     |1      |1      |Aosta |31       |188          |126957      |126957      |[Aosta]     |[2010/2011]         |(4,[],[])           |(4,[1],[1.0])               |
|2011/2012     |1      |1      |Aosta |21       |172          |127153      |127153      |[Aosta]     |[2011/2012]         |(4,[],[])           |(4,[],[])                   |
|2012/2013     |1      |1      |Aosta |31       |173          |127305      |127305      |[Aosta]     |[2012/2013]         |(4,[],[

In [ ]:
df_ohe=df_ohe.drop('Ateneo')
df_ohe=df_ohe.drop('AnnoAccademico')
df_ohe=df_ohe.drop('Ateneo_Array')
df_ohe=df_ohe.drop('AnnoAccademico_Array')

In [ ]:
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml.feature import StandardScaler

columns=df_ohe.columns
columns.remove('Immatricolati')
assembler=VectorAssembler(inputCols=columns, outputCol='features_prescaled')
df=assembler.transform(df_ohe)

# Normalizing each feature to have unit standard deviation
scaler = StandardScaler(inputCol="features_prescaled", outputCol="features", withStd=True, withMean=False)
scalerModel = scaler.fit(df)
# Normalize each feature to have unit standard deviation.
df = scalerModel.transform(df)


stringIndexer=StringIndexer(inputCol='Immatricolati',outputCol='label')
si_model=stringIndexer.fit(df)
df=si_model.transform(df)

In [ ]:
df=df.drop('features_prescaled')
df.show()

+-------+-------+---------+-------------+------------+------------+--------------------+----------------------------+--------------------+-----+
|AteneiP|AteneiR|Fuorisede|Immatricolati|PopolazioneP|PopolazioneR|Ateneo_OneHotEncoded|AnnoAccademico_OneHotEncoded|            features|label|
+-------+-------+---------+-------------+------------+------------+--------------------+----------------------------+--------------------+-----+
|      1|      1|       31|          188|      126957|      126957|           (4,[],[])|               (4,[1],[1.0])|(13,[0,1,2,3,4,10...|173.0|
|      1|      1|       21|          172|      127153|      127153|           (4,[],[])|                   (4,[],[])|(13,[0,1,2,3,4],[...|157.0|
|      1|      1|       31|          173|      127305|      127305|           (4,[],[])|                   (4,[],[])|(13,[0,1,2,3,4],[...|159.0|
|      1|      1|       54|          192|      127951|      127951|           (4,[],[])|                   (4,[],[])|(13,[0,1,2,3,

In [ ]:
#trainingData=df.sampleBy('label', fractions={0:0.7,1:0.7,2: 0.7}, seed=100)
#testData=df.subtract(trainingData)

#Split

In [ ]:
df_ml=df

In [ ]:
(trainingData, testData) = df_ml.randomSplit([0.75, 0.25])
(partialTrainingData, validationData) = trainingData.randomSplit([0.75, 0.25])

#Linear regressor


In [ ]:
from pyspark.ml.regression import LinearRegression

linear_regressor = LinearRegression(featuresCol = 'features', labelCol = 'label')

#Learn to fit the model from training set
regressor = linear_regressor.fit(partialTrainingData)

#To predict the prices on testing set
pred = regressor.evaluate(validationData)

#Predict the model
pred.predictions.show()

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------+-------+---------+-------------+------------+------------+--------------------+----------------------------+--------------------+-----+------------------+
|AteneiP|AteneiR|Fuorisede|Immatricolati|PopolazioneP|PopolazioneR|Ateneo_OneHotEncoded|AnnoAccademico_OneHotEncoded|            features|label|        prediction|
+-------+-------+---------+-------------+------------+------------+--------------------+----------------------------+--------------------+-----+------------------+
|      1|      1|       31|          173|      127305|      127305|           (4,[],[])|                   (4,[],[])|(13,[0,1,2,3,4],[...|159.0|  310.877691403365|
|      1|      1|       54|          192|      127951|      127951|           (4,[],[])|                   (4,[],[])|(13,[0,1,2,3,4],[...|  0.0| 310.7237868492553|
|      1|      1|       70|          185|      127972|      127972|           (4,[],[])|               (4,[3],[1.0])|(13,[0,1,2,3,4,12...|  8.0|319.83710241691944|
|      1|      1

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


RMSE: 187.342
MSE: 35097.094
MAE: 159.654
r2: -0.021


test


In [ ]:
from pyspark.ml.regression import LinearRegression

linear_regressor = LinearRegression(featuresCol = 'features', labelCol = 'label')

#Learn to fit the model from training set
regressor = linear_regressor.fit(partialTrainingData)

#To predict the prices on testing set
pred = regressor.evaluate(testData)

#Predict the model
pred.predictions.show()

from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------+-------+---------+-------------+------------+------------+--------------------+----------------------------+--------------------+-----+------------------+
|AteneiP|AteneiR|Fuorisede|Immatricolati|PopolazioneP|PopolazioneR|Ateneo_OneHotEncoded|AnnoAccademico_OneHotEncoded|            features|label|        prediction|
+-------+-------+---------+-------------+------------+------------+--------------------+----------------------------+--------------------+-----+------------------+
|      1|      1|       31|          188|      126957|      126957|           (4,[],[])|               (4,[1],[1.0])|(13,[0,1,2,3,4,10...|173.0| 291.4931916150157|
|      1|      1|       47|          166|      127030|      127030|           (4,[],[])|               (4,[0],[1.0])|(13,[0,1,2,3,4,9]...|148.0|357.86035078004204|
|      1|      1|       59|          185|      126677|      126677|           (4,[],[])|                   (4,[],[])|(13,[0,1,2,3,4],[...|  8.0| 310.7085623172335|
|      1|      1

Train - Test


In [ ]:
from pyspark.ml.regression import LinearRegression
regressor = LinearRegression(featuresCol = 'features', labelCol = 'label')

linear_regressor = LinearRegression(featuresCol = 'features', labelCol = 'label')

#Learn to fit the model from training set
regressor = linear_regressor.fit(trainingData)

#To predict the prices on testing set
pred = regressor.evaluate(validationData)

#Predict the model
pred.predictions.show()

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------+-------+---------+-------------+------------+------------+--------------------+----------------------------+--------------------+-----+------------------+
|AteneiP|AteneiR|Fuorisede|Immatricolati|PopolazioneP|PopolazioneR|Ateneo_OneHotEncoded|AnnoAccademico_OneHotEncoded|            features|label|        prediction|
+-------+-------+---------+-------------+------------+------------+--------------------+----------------------------+--------------------+-----+------------------+
|      1|      1|       31|          188|      126957|      126957|           (4,[],[])|               (4,[1],[1.0])|(13,[0,1,2,3,4,10...|173.0|290.43018123661557|
|      1|      1|       47|          166|      127030|      127030|           (4,[],[])|               (4,[0],[1.0])|(13,[0,1,2,3,4,9]...|148.0|342.85855379670244|
|      1|      1|       59|          185|      126677|      126677|           (4,[],[])|                   (4,[],[])|(13,[0,1,2,3,4],[...|  8.0|292.60501927110386|
|      1|      1

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


RMSE: 189.441
MSE: 35888.034
MAE: 163.081
r2: -0.024


#Random Forest Regressor


In [ ]:
from pyspark.ml.regression import RandomForestRegressor

lr = RandomForestRegressor(featuresCol = 'features', labelCol = 'label', numTrees=1000, maxDepth=3)
lr_model = lr.fit(partialTrainingData)

predictions = lr_model.transform(validationData)
predictions.select("prediction", "label", "features").show()

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 274.0585907430483|159.0|(13,[0,1,2,3,4],[...|
| 274.0585907430483|  0.0|(13,[0,1,2,3,4],[...|
| 281.5848426437429|  8.0|(13,[0,1,2,3,4,12...|
|311.15045983005314|603.0|(13,[0,1,2,3,4],[...|
|311.15045983005314| 39.0|(13,[0,1,2,3,4],[...|
| 293.7340611858891| 61.0|(13,[0,1,2,3,4],[...|
| 279.3264794660861| 43.0|(13,[0,1,2,3,4],[...|
|  296.812426851769|305.0|(13,[0,1,2,3,4],[...|
| 300.9456865510076|325.0|(13,[0,1,2,3,4,12...|
| 300.4985650105165| 19.0|(13,[0,1,2,3,4],[...|
| 296.4010807330616|287.0|(13,[0,1,2,3,4],[...|
|299.98789402423955|274.0|(13,[0,1,2,3,4,12...|
|  308.709538657708|401.0|(13,[0,1,2,3,4,10...|
|  315.020433341009|448.0|(13,[0,1,2,3,4,9]...|
|326.45977756797066|469.0|(13,[0,1,2,3,4],[...|
| 320.9462273709988|490.0|(13,[0,1,2,3,4,10...|
|290.52200280703914|155.0|(13,[0,1,2,3,4,12...|
|282.20656320256046|586.0|(13,[0,1,2,3,4

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

eval = RegressionEvaluator(labelCol="label",predictionCol="prediction", metricName="rmse")
rmse = eval.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g"% rmse)

mse = eval.evaluate(predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

Root Mean Squared Error (RMSE) on test data = 169.576
MSE: 28755.868
MAE: 139.521
r2: 0.164


Test

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

lr = RandomForestRegressor(featuresCol = 'features', labelCol = 'label', numTrees=1000, maxDepth=3)
lr_model = lr.fit(partialTrainingData)

predictions = lr_model.transform(testData)
predictions.select("prediction", "label", "features").show()

from pyspark.ml.evaluation import RegressionEvaluator

eval = RegressionEvaluator(labelCol="label",predictionCol="prediction", metricName="rmse")
rmse = eval.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g"% rmse)

mse = eval.evaluate(predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|265.85041574243434|173.0|(13,[0,1,2,3,4,10...|
| 285.3699179877601|148.0|(13,[0,1,2,3,4,9]...|
| 274.0585907430483|  8.0|(13,[0,1,2,3,4],[...|
|311.15045983005314|614.0|(13,[0,1,2,3,4],[...|
| 311.6605093495396|610.0|(13,[0,1,2,3,4],[...|
| 279.3264794660861|  1.0|(13,[0,1,2,3,4],[...|
| 290.7728283055689|617.0|(13,[0,1,2,3,4,9]...|
|  296.812426851769|320.0|(13,[0,1,2,3,4],[...|
|309.70345494697057|371.0|(13,[0,1,2,3,4],[...|
| 292.9294658912709|273.0|(13,[0,1,2,3,4],[...|
| 325.8460744509479|458.0|(13,[0,1,2,3,4],[...|
| 333.3056615385524|546.0|(13,[0,1,2,3,4],[...|
| 286.4970181029444|  6.0|(13,[0,1,2,3,4],[...|
| 286.1426995946726|174.0|(13,[0,1,2,3,4],[...|
| 286.5741054996205| 92.0|(13,[0,1,2,3,4],[...|
| 285.4173222824171|  7.0|(13,[0,1,2,3,4],[...|
|282.37892815051583|115.0|(13,[0,1,2,3,4,10...|
| 280.3685492416668|612.0|(13,[0,1,2,3,4

Train - Test

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

lr = RandomForestRegressor(featuresCol = 'features', labelCol = 'label', numTrees=1000, maxDepth=3)
lr_model = lr.fit(trainingData)

predictions = lr_model.transform(testData)
predictions.select("prediction", "label", "features").show()

from pyspark.ml.evaluation import RegressionEvaluator

eval = RegressionEvaluator(labelCol="label",predictionCol="prediction", metricName="rmse")
rmse = eval.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g"% rmse)

mse = eval.evaluate(predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 241.9072058340789|173.0|(13,[0,1,2,3,4,10...|
|257.67790959732787|148.0|(13,[0,1,2,3,4,9]...|
|244.77376845568318|  8.0|(13,[0,1,2,3,4],[...|
| 295.7808574250489|614.0|(13,[0,1,2,3,4],[...|
| 295.7808574250489|610.0|(13,[0,1,2,3,4],[...|
| 257.0370147820624|  1.0|(13,[0,1,2,3,4],[...|
|269.51229970502646|617.0|(13,[0,1,2,3,4,9]...|
|291.89186272535204|320.0|(13,[0,1,2,3,4],[...|
| 299.2724701917623|371.0|(13,[0,1,2,3,4],[...|
| 293.3922005817819|273.0|(13,[0,1,2,3,4],[...|
| 327.8309405427998|458.0|(13,[0,1,2,3,4],[...|
| 330.4195407144828|546.0|(13,[0,1,2,3,4],[...|
|287.72836455291525|  6.0|(13,[0,1,2,3,4],[...|
| 287.7743406948719|174.0|(13,[0,1,2,3,4],[...|
| 288.0336276995334| 92.0|(13,[0,1,2,3,4],[...|
| 287.7743406948719|  7.0|(13,[0,1,2,3,4],[...|
|285.86932777815497|115.0|(13,[0,1,2,3,4,10...|
|278.91350140455967|612.0|(13,[0,1,2,3,4

#GBT Regressor

In [ ]:
from pyspark.ml.regression import GBTRegressor

lr = GBTRegressor(featuresCol = "features", labelCol="label", maxIter=100, stepSize=0.01)
lr_model = lr.fit(partialTrainingData)

predictions = lr_model.transform(validationData)
predictions.select("prediction", "label", "features").show()

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 202.2495481870274|159.0|(13,[0,1,2,3,4],[...|
| 202.2495481870274|  0.0|(13,[0,1,2,3,4],[...|
| 93.17162787553885|  8.0|(13,[0,1,2,3,4,12...|
|449.42407502027453|603.0|(13,[0,1,2,3,4],[...|
|449.42407502027453| 39.0|(13,[0,1,2,3,4],[...|
| 305.6307953978314| 61.0|(13,[0,1,2,3,4],[...|
| 227.5381987530288| 43.0|(13,[0,1,2,3,4],[...|
|328.94494112714773|305.0|(13,[0,1,2,3,4],[...|
|355.61441104394135|325.0|(13,[0,1,2,3,4,12...|
|337.31391072116406| 19.0|(13,[0,1,2,3,4],[...|
|271.17206009428054|287.0|(13,[0,1,2,3,4],[...|
|297.84153001107404|274.0|(13,[0,1,2,3,4,12...|
|267.12836277785414|401.0|(13,[0,1,2,3,4,10...|
|  284.587501669394|448.0|(13,[0,1,2,3,4,9]...|
| 311.2182032480407|469.0|(13,[0,1,2,3,4],[...|
|  236.196044393777|490.0|(13,[0,1,2,3,4,10...|
|262.38872517785296|155.0|(13,[0,1,2,3,4,12...|
| 392.0130641645033|586.0|(13,[0,1,2,3,4

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

eval = RegressionEvaluator(labelCol="label",predictionCol="prediction", metricName="rmse")
rmse = eval.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g"% rmse)

mse = eval.evaluate(predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

Root Mean Squared Error (RMSE) on test data = 157.987
MSE: 24959.793
MAE: 110.798
r2: 0.274


Test

In [ ]:
from pyspark.ml.regression import GBTRegressor

lr = GBTRegressor(featuresCol = "features", labelCol="label", maxIter=100, stepSize=0.01)
lr_model = lr.fit(trainingData)

predictions = lr_model.transform(validationData)
predictions.select("prediction", "label", "features").show()

from pyspark.ml.evaluation import RegressionEvaluator

eval = RegressionEvaluator(labelCol="label",predictionCol="prediction", metricName="rmse")
rmse = eval.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g"% rmse)

mse = eval.evaluate(predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 149.5591074120967|159.0|(13,[0,1,2,3,4],[...|
| 149.5591074120967|  0.0|(13,[0,1,2,3,4],[...|
| 79.30172406755689|  8.0|(13,[0,1,2,3,4,12...|
|401.27563075300645|603.0|(13,[0,1,2,3,4],[...|
|401.27563075300645| 39.0|(13,[0,1,2,3,4],[...|
| 218.8222047185386| 61.0|(13,[0,1,2,3,4],[...|
|145.42134626237498| 43.0|(13,[0,1,2,3,4],[...|
|295.55577133662916|305.0|(13,[0,1,2,3,4],[...|
| 344.1818518071412|325.0|(13,[0,1,2,3,4,12...|
| 300.8510342846959| 19.0|(13,[0,1,2,3,4],[...|
|284.10487735159785|287.0|(13,[0,1,2,3,4],[...|
|285.67628667837613|274.0|(13,[0,1,2,3,4,12...|
| 377.0989897222215|401.0|(13,[0,1,2,3,4,10...|
|379.72806164146294|448.0|(13,[0,1,2,3,4,9]...|
| 363.3826616156749|469.0|(13,[0,1,2,3,4],[...|
| 364.6176296156749|490.0|(13,[0,1,2,3,4,10...|
|250.36068787730167|155.0|(13,[0,1,2,3,4,12...|
| 370.3689124746325|586.0|(13,[0,1,2,3,4

In [ ]:
from pyspark.ml.regression import GBTRegressor

lr = GBTRegressor(featuresCol = "features", labelCol="label", maxIter=100, stepSize=0.01)
lr_model = lr.fit(trainingData)

predictions = lr_model.transform(testData)
predictions.select("prediction", "label", "features").show()

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|117.33135630749027|173.0|(13,[0,1,2,3,4,10...|
| 205.0140761141448|148.0|(13,[0,1,2,3,4,9]...|
| 149.5591074120967|  8.0|(13,[0,1,2,3,4],[...|
|401.27563075300645|614.0|(13,[0,1,2,3,4],[...|
|401.27563075300645|610.0|(13,[0,1,2,3,4],[...|
|145.42134626237498|  1.0|(13,[0,1,2,3,4],[...|
|222.63648455806182|617.0|(13,[0,1,2,3,4,9]...|
|295.55577133662916|320.0|(13,[0,1,2,3,4],[...|
|348.10086423760845|371.0|(13,[0,1,2,3,4],[...|
|  259.869295034826|273.0|(13,[0,1,2,3,4],[...|
| 362.0730036578763|458.0|(13,[0,1,2,3,4],[...|
|477.22077102840234|546.0|(13,[0,1,2,3,4],[...|
| 245.7922610840328|  6.0|(13,[0,1,2,3,4],[...|
| 245.7922610840328|174.0|(13,[0,1,2,3,4],[...|
| 245.7922610840328| 92.0|(13,[0,1,2,3,4],[...|
| 245.7922610840328|  7.0|(13,[0,1,2,3,4],[...|
|205.62498145766025|115.0|(13,[0,1,2,3,4,10...|
|174.35387148265613|612.0|(13,[0,1,2,3,4

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

eval = RegressionEvaluator(labelCol="label",predictionCol="prediction", metricName="rmse")
rmse = eval.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g"% rmse)

mse = eval.evaluate(predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

Root Mean Squared Error (RMSE) on test data = 127.022
MSE: 16134.544
MAE: 87.522
r2: 0.540


#TRAINING FINALE


In [ ]:
lr = GBTRegressor(featuresCol = "features", labelCol="label", maxIter=100, stepSize=0.01)
lr_model = lr.fit(df)

#Testing

In [ ]:
with open("/content/drive/MyDrive/KPMG/RegressionDatasetTest_finale.xlsx", 'rb') as fp:
    df = pd.read_excel(fp)

In [ ]:
df=df.to_dict('records')

In [ ]:
import json

path = "/content/drive/MyDrive/KPMG/"

with open(path + 'RegressionDataset_Test.json', 'w') as jsonfile:
    json.dump(df, jsonfile)

In [ ]:
# Read JSON file into dataframe
path = "/content/drive/MyDrive/KPMG/"

df_spark = spark.read.json(path + 'RegressionDataset_Test.json')
df_spark.printSchema()
df_spark.show()

root
 |-- AnnoAccademico: string (nullable = true)
 |-- AteneiP: long (nullable = true)
 |-- AteneiR: long (nullable = true)
 |-- Ateneo: string (nullable = true)
 |-- FuoriSede: long (nullable = true)
 |-- Immatricolati: long (nullable = true)
 |-- PopolazioneP: long (nullable = true)
 |-- PopolazioneR: long (nullable = true)

+--------------+-------+-------+----------------+---------+-------------+------------+------------+
|AnnoAccademico|AteneiP|AteneiR|          Ateneo|FuoriSede|Immatricolati|PopolazioneP|PopolazioneR|
+--------------+-------+-------+----------------+---------+-------------+------------+------------+
|     2021/2022|      1|      1|           Aosta|       77|          246|      123041|      123704|
|     2022/2023|      1|      1|           Aosta|       81|          248|      122745|      123408|
|     2023/2024|      1|      1|           Aosta|       85|          250|      122449|      123112|
|     2024/2025|      1|      1|           Aosta|       89|          2

In [ ]:
df_spark=df_spark.na.drop()

In [ ]:
from pyspark.sql.functions import col, split
df = df_spark.withColumn("Ateneo_Array", split(col("Ateneo")," "))
df.show()

+--------------+-------+-------+----------------+---------+-------------+------------+------------+-------------------+
|AnnoAccademico|AteneiP|AteneiR|          Ateneo|FuoriSede|Immatricolati|PopolazioneP|PopolazioneR|       Ateneo_Array|
+--------------+-------+-------+----------------+---------+-------------+------------+------------+-------------------+
|     2021/2022|      1|      1|           Aosta|       77|          246|      123041|      123704|            [Aosta]|
|     2022/2023|      1|      1|           Aosta|       81|          248|      122745|      123408|            [Aosta]|
|     2023/2024|      1|      1|           Aosta|       85|          250|      122449|      123112|            [Aosta]|
|     2024/2025|      1|      1|           Aosta|       89|          252|      122153|      122816|            [Aosta]|
|     2025/2026|      1|      1|           Aosta|       93|          254|      121857|      122520|            [Aosta]|
|     2021/2022|      2|      3|        

In [ ]:
from pyspark.sql.functions import col, split
df = df_spark.withColumn("Ateneo_Array", split(col("Ateneo")," "))
df = df.withColumn('AnnoAccademico_Array', split(col("AnnoAccademico")," "))
df.show()

#Import Spark CountVectorizer
from pyspark.ml.feature import CountVectorizer

# Initialize a CountVectorizer.
colorVectorizer = CountVectorizer(inputCol="Ateneo_Array", outputCol="Ateneo_OneHotEncoded", vocabSize=4, minDF=1.0)

#Get a VectorizerModel
colorVectorizer_model = colorVectorizer.fit(df)

df_ohe = colorVectorizer_model.transform(df)
df_ohe.show(truncate=False)



+--------------+-------+-------+----------------+---------+-------------+------------+------------+-------------------+--------------------+
|AnnoAccademico|AteneiP|AteneiR|          Ateneo|FuoriSede|Immatricolati|PopolazioneP|PopolazioneR|       Ateneo_Array|AnnoAccademico_Array|
+--------------+-------+-------+----------------+---------+-------------+------------+------------+-------------------+--------------------+
|     2021/2022|      1|      1|           Aosta|       77|          246|      123041|      123704|            [Aosta]|         [2021/2022]|
|     2022/2023|      1|      1|           Aosta|       81|          248|      122745|      123408|            [Aosta]|         [2022/2023]|
|     2023/2024|      1|      1|           Aosta|       85|          250|      122449|      123112|            [Aosta]|         [2023/2024]|
|     2024/2025|      1|      1|           Aosta|       89|          252|      122153|      122816|            [Aosta]|         [2024/2025]|
|     2025/20

In [ ]:
#Import Spark CountVectorizer
from pyspark.ml.feature import CountVectorizer

# Initialize a CountVectorizer.
colorVectorizer = CountVectorizer(inputCol="AnnoAccademico_Array", outputCol="AnnoAccademico_OneHotEncoded", vocabSize=4, minDF=1.0)

#Get a VectorizerModel
colorVectorizer_model = colorVectorizer.fit(df)

df_ohe = colorVectorizer_model.transform(df_ohe)
df_ohe.show(truncate=False)

+--------------+-------+-------+----------------+---------+-------------+------------+------------+-------------------+--------------------+--------------------+----------------------------+
|AnnoAccademico|AteneiP|AteneiR|Ateneo          |FuoriSede|Immatricolati|PopolazioneP|PopolazioneR|Ateneo_Array       |AnnoAccademico_Array|Ateneo_OneHotEncoded|AnnoAccademico_OneHotEncoded|
+--------------+-------+-------+----------------+---------+-------------+------------+------------+-------------------+--------------------+--------------------+----------------------------+
|2021/2022     |1      |1      |Aosta           |77       |246          |123041      |123704      |[Aosta]            |[2021/2022]         |(4,[],[])           |(4,[1],[1.0])               |
|2022/2023     |1      |1      |Aosta           |81       |248          |122745      |123408      |[Aosta]            |[2022/2023]         |(4,[],[])           |(4,[2],[1.0])               |
|2023/2024     |1      |1      |Aosta        

In [ ]:
df_ohe=df_ohe.drop('Ateneo')
df_ohe=df_ohe.drop('AnnoAccademico')
df_ohe=df_ohe.drop('Ateneo_Array')
df_ohe=df_ohe.drop('AnnoAccademico_Array')

In [ ]:
from pyspark.ml.feature import VectorAssembler,StringIndexer

columns=df_ohe.columns
columns.remove('Immatricolati')
assembler=VectorAssembler(inputCols=columns, outputCol='features_prescaled')
df=assembler.transform(df_ohe)

# Normalizing each feature to have unit standard deviation
scaler = StandardScaler(inputCol="features_prescaled", outputCol="features", withStd=True, withMean=False)
scalerModel = scaler.fit(df)
# Normalize each feature to have unit standard deviation.
df = scalerModel.transform(df)

stringIndexer=StringIndexer(inputCol='Immatricolati',outputCol='label')
si_model=stringIndexer.fit(df)
df=si_model.transform(df)

In [ ]:
predictions = lr_model.transform(df)
predictions.select("prediction", "label", "features").show()

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|54.383610330079485| 91.0|(13,[0,1,2,3,4,10...|
| 185.4469393195335| 93.0|(13,[0,1,2,3,4,11...|
| 51.94209601468587| 95.0|(13,[0,1,2,3,4,9]...|
| 51.94209601468587| 96.0|(13,[0,1,2,3,4],[...|
|  52.9577491748624| 98.0|(13,[0,1,2,3,4,12...|
|  529.555755325561|250.0|(13,[0,1,2,3,4,10...|
| 525.2748927221161|246.0|(13,[0,1,2,3,4,11...|
| 526.9773614497258|242.0|(13,[0,1,2,3,4,9]...|
| 525.2748927221161|241.0|(13,[0,1,2,3,4],[...|
| 513.9923196845755|239.0|(13,[0,1,2,3,4,12...|
|184.86776682960377| 63.0|(13,[0,1,2,3,4,8,...|
|200.29802181539995| 65.0|(13,[0,1,2,3,4,8,...|
|203.14213626700317| 67.0|(13,[0,1,2,3,4,8,...|
|212.45349987464326|  0.0|(13,[0,1,2,3,4,8]...|
| 222.8647988864094| 69.0|(13,[0,1,2,3,4,8,...|
| 134.2547326256147|281.0|(13,[0,1,2,3,4,10...|
|233.60602096569392|279.0|(13,[0,1,2,3,4,11...|
|227.15458328400751|278.0|(13,[0,1,2,3,4